In [ ]:
import configparser
import getpass
import os
import requests

In [ ]:
os.environ['GITHUB_USER'] = input('Github Username: ')

In [ ]:
os.environ['GITHUB_API_KEY'] = getpass.getpass(prompt='GitHubAPI Key: ', stream=None)

# Github API Requests

In this notebook, we'll play a little bit with the [GitHub API](https://developer.github.com/v3/). When you are working with _any_ API, please pay attention to any [rate limiting](https://developer.github.com/v3/#rate-limiting) they have in place. Often providers will have different rate limits depending on whether or not you are authenticated.

The two cells above will prompt you to put your some github accounts details. If you don't already have a GitHub API key, you can generate one for yourself [From here](https://github.com/settings/tokens). You should treat API keys securely, if they are leaked the will allow people to (ab)use your account. The nice thing about API keys is you can usually restrict how the can be used and you can delete them when they've served their purpose. 

For this notebook, we'll put our API keys into environment variables. These will be destroyed when our session ends.  If you just stored your keys as a python variables they would be storead along with your code, when you share your notebook with someone or post it to github, people would have access to your keys. More generally, it is a good idea to separate configuration from code and environment variables is one good way of doing that.

Setting the variables every time you open this notebook can be irritating, but this is an _interactive_ tutorial, so we'll live with it. If you end up doing this a lot, there are modules (e.g. [configparser](https://docs.python.org/3/library/configparser.html)) which can help you manage them as files. Just make sure you set secure permissions on those files and never store them under version control (it's too easy to push to github!)

In [ ]:
assert len(os.environ['GITHUB_USER']) > 0, "Unset GitHub Username"
assert len(os.environ['GITHUB_API_KEY']) > 0, "Unset GitHub Password"

In [ ]:
payload = {'q': 'nbformat_minor+extension:ipynb'}
resp = requests.get(
    'https://api.github.com/search/code',
    params=payload,
    auth=(os.environ.get('GITHUB_USER'), os.environ.get('GITHUB_API_KEY'))
)
resp.raise_for_status()


In [ ]:
resp.json()['total_count']

Actually, [Peter Parente](`https://github.com/parente`) has been tracking this quantity for years, his data is available as a CSV, so with a little bit of pandas...

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
nbHitsDF = pd.read_csv(
    'https://raw.githubusercontent.com/parente/nbestimate/master/ipynb_counts.csv',
    parse_dates=['date'],
    index_col='date'
)
nbHitsDF.tail()

In [ ]:
nbHitsDF.plot(
    markersize=10,
    marker='+',
    figsize=(16,10)
)

Looks like an exponential to me...

In [ ]:
nbHitsDF['logHits'] = np.log(nbHitsDF['hits'])
nbHitsDF['logHits'].plot(figsize=(16,10))

OK, maybe just a little bit less than exponential, but still! There are probably better ways of doing this, but...

In [ ]:
nbHitsDF['sinceStart'] = (nbHitsDF.index - nbHitsDF.index[0]).days

In [ ]:
params = np.polyfit(nbHitsDF['sinceStart'], nbHitsDF['logHits'], 2)

params

In [ ]:
fig = plt.figure(figsize=(16,10))

ax = fig.add_subplot(1, 1, 1)
ax.plot(nbHitsDF['sinceStart'], nbHitsDF['logHits'])
ax.plot(
    nbHitsDF['sinceStart'], 
    params[2] + nbHitsDF['sinceStart'] * (params[1] + nbHitsDF['sinceStart'] * params[0])
)

**TODO**: Try to figure out when GitHub will have more than 8M notebooks

## User endpoint

The endpoint for user repositories is https://api.github.com/users/<username>/repos. Let's see what repositories we can find, (try your own github username if you like)

In [ ]:
payload = {'sort': 'push'}
resp = requests.get(
    'https://api.github.com/users/torvalds/repos',
    params=payload,
    auth=(os.environ.get('GITHUB_USER'), os.environ.get('GITHUB_API_KEY'))
)
resp.raise_for_status()

In [ ]:
torvaldsDF = pd.read_json(resp.text)
torvaldsDF

**TODO**: See if you can extract linus' public key!

In [ ]:
resp = requests.get(
    'https://api.github.com/users/torvalds/keys',
    auth=(os.environ.get('GITHUB_USER'), os.environ.get('GITHUB_API_KEY'))
)
resp.raise_for_status()
pd.read_json(resp.text)

## Other Endpoints

There are [lots of other interesting endpoints](https://developer.github.com/v3) to play with, here are some of the more interesting ones:

  * [Collaborators](https://developer.github.com/v3/repos/collaborators/)
  * [Repositories](https://developer.github.com/v3/repos) - List repos, create new ones etc.
    * [Commits](https://developer.github.com/v3/repos/commits/) - examine past commits
    * [Statistics](https://developer.github.com/v3/repos/statistics/) - repository stats, who commits most often etc.
  * [Search](https://developer.github.com/v3/search/) - Find all the data
    * [Commits](https://developer.github.com/v3/search/#search-commits)
    * [Users](https://developer.github.com/v3/search/#search-users)
    * [Repositories](https://developer.github.com/v3/search/#search-repositories)
    * [Issues](https://developer.github.com/v3/search/#search-issues-and-pull-requests)
  * [Pull Requests](https://developer.github.com/v3/pulls/) - Creation and merge PRs automatically!
  
  
I'd _strongly_ recommend sticking to the GET methods until you are comfortable, but once you are ready, the (requests)[https://2.python-requests.org/en/master/] module has all of the options you need to make POST/PUT/DELETE requests to make changes.
